In [ ]:
import sys
sys.path.append("../input/ttach-kaggle/")
sys.path.append("../input/timmmaster/")
sys.path.append("../input/bbbbox/")
sys.path.append("../input/pytorchlighting/")



In [ ]:

import os
import warnings
from pprint import pprint
from glob import glob
from tqdm import tqdm

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchvision.transforms as T
from box import Box
from timm import create_model
from sklearn.model_selection import StratifiedKFold
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset
# from pytorch_grad_cam import GradCAMPlusPlus
# from pytorch_grad_cam.utils.image import show_cam_on_image


import pytorch_lightning as pl
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning import callbacks
from pytorch_lightning.callbacks.progress import ProgressBarBase
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import LightningDataModule, LightningModule


warnings.filterwarnings("ignore")



config = {'seed': 2022,
          'root': "../input/petfinder-pawpularity-score", 
          'n_splits': 5,
          'epoch': 10,
          'trainer': {
              'gpus': 1,
              'accumulate_grad_batches': 1,
              'progress_bar_refresh_rate': 1,
              'fast_dev_run': False,
              'num_sanity_val_steps': 0,
              'resume_from_checkpoint': None,
          },
          'transform':{
              'name': 'get_default_transforms',
              'image_size': 224
          },
          'train_loader':{
              'batch_size': 64,
              'shuffle': True,
              'num_workers': 4,
              'pin_memory': False,
              'drop_last': True,
          },
          'val_loader': {
              'batch_size': 64,
              'shuffle': False,
              'num_workers': 4,
              'pin_memory': False,
              'drop_last': False
          },
          'model':{
              'name': 'swin_tiny_patch4_window7_224',
              'output_dim': 1
          },
          'optimizer':{
              'name': 'optim.AdamW',
              'params':{
                  'lr': 1e-5
              },
          },
          'scheduler':{
              'name': 'optim.lr_scheduler.CosineAnnealingWarmRestarts',
              'params':{
                  'T_0': 20,
                  'eta_min': 1e-4,
              }
          },
          'loss': 'nn.BCEWithLogitsLoss',
}

config = Box(config)

config = Box(config)

class PetfinderDataset(Dataset):
    def __init__(self, df, image_size=224):
        self._X = df["Path"].values
        self._y = None
        if "Pawpularity" in df.keys():
            self._y = df["Pawpularity"].values
        self._transform = T.Resize([image_size, image_size])

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        image_path = self._X[idx]
        image = read_image(image_path)
        image = self._transform(image)
        if self._y is not None:
            label = self._y[idx]
            return image, label
        return image



torch.autograd.set_detect_anomaly(True)
seed_everything(config.seed)


IMAGENET_MEAN = [0.485, 0.456, 0.406]  # RGB
IMAGENET_STD = [0.229, 0.224, 0.225]  # RGB


def get_default_transforms():
    transform = {
        "train": T.Compose(
            [
                T.RandomHorizontalFlip(),
                T.RandomVerticalFlip(),
                T.RandomAffine(15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
                T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
            ]
        ),
        "val": T.Compose(
            [
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
            ]
        ),
    }
    return transform




class Model(pl.LightningModule):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.__build_model()
        self._criterion = eval(self.cfg.loss)()
        self.transform = get_default_transforms()
        self.save_hyperparameters(cfg)

    def __build_model(self):
        self.backbone = create_model(
            self.cfg.model.name, pretrained=False, num_classes=0, in_chans=3
        )
        num_features = self.backbone.num_features
        self.fc = nn.Sequential(
            nn.Dropout(0.5), nn.Linear(num_features, self.cfg.model.output_dim)
        )

    def forward(self, x):
        f = self.backbone(x)
        out = self.fc(f)
        return out
    
    def predict_step(self, batch, batch_idx):
        images = batch
        images = self.transform['val'](images)
    
        logits = self.forward(images).squeeze(1)
        
        pred = logits.sigmoid().detach().cpu() * 100.
        return pred

swin_trans_preds = []    
model_folder = "../input/swint224"
df = pd.read_csv(os.path.join(config.root, "test.csv"))
df["Path"] = df["Id"].apply(lambda x: os.path.join(config.root, "test", x + ".jpg"))

for i in range(10):
    model = Model(config)
    checkpoint_dir = f'../input/swint224/model_f'+str(i)+'.ckpt'
    model.load_state_dict(torch.load(checkpoint_dir)['state_dict'])
    trainer_params = {"gpus": 1}
    trainer = pl.Trainer(**trainer_params)
    ds = DataLoader(PetfinderDataset(df), shuffle=False, batch_size=1)
    prediction = trainer.predict(model, ds)
    swin_trans_preds.append(prediction)
print(swin_trans_preds)
predictions = np.mean(swin_trans_preds, axis=0)
sub_csv = pd.DataFrame({
    "Id": df.Id,
    "Pawpularity": predictions
}) 


In [ ]:
sub_csv.to_csv("submission.csv", index=False)
pd.read_csv("submission.csv").head()